# Projeto de Inteligência Artificial (Parte 1)

### Realizado pelo Grupo "NoMaidens"
$\rightarrow$ Diogo Araújo - a21905661 <br>
$\rightarrow$ Miguel Pereira - a21902726 <br>

## Introdução
O objetivo deste projeto foi a implementação de um algoritmo de **pesquisa informada** num ambiente estático com obstáculos capaz de encontrar um caminho válido e de menor custo desde o **start node** até o **goal node**. <br>

### Nós e Obstáculos
O ambiente que iremos utilizar é um mundo quadrado representado através duma grelha 100x100. Esta será representada por 6 tipos diferentes de nós: <br>

$\rightarrow$ Temos o **nó inicial** (S) e o **nó final** (E), que estão no canto superior esquerdo e no canto inferior direito da grelha, respetivamente, e representam o ínicio e fim do algoritmo; <br>

$\rightarrow$ Temos os nós em que nos podemos movimentar, sendo esses a **terra** (T) e a **água** (A) cuja movimentação para eles tem um custo de 1 e 3, respetivamente; <br>

$\rightarrow$ Temos os nós que servem como obstáculos, sendo esses a **barreira** (B) e a **fronteira** (F), ou seja que impedem a movimentação para esses nós; <br>

Os obstáculos também têm diferentes formatos/tipos que podem ser sobrepostos entre si. No nosso caso, o algoritmo não tomou partido desta informação devido a não acharmos devidamente utilizável no algoritmo pois estes podiam estar sobrepostos, retirando a possibilidade de efetuar conclusões a partir dos obstáculos que eram encontrados. <br>

$\rightarrow$ Existem os **lagos**, que têm uma dimensão 2x2 e são compostos por água. <br>

$\rightarrow$ Existem as **paredes**, que têm dimensões de 1x10 ou 10x1 e são compostos por barreiras. <br>

$\rightarrow$ Existe também o **limite do ambiente**, que irá delimitar todo o ambiente sem se sobrepor ao nó inicial e final e que é composto por fronteiras. <br>

### Movimentação
Serão considerados como **filhos** de um nó todos aqueles que são **adjacentes**. Os filhos são todos os nós que o nosso algoritmo irá visualizar e poderá movimentar-se para. <br> 
Um exemplo desta movimentação seria o Rei, no Xadrez, que pode movimentar-se para todas as posições à sua volta. <br>

### Ambiente
O **ambiente** será retirado a partir de um **ficheiro** ".csv" que no nosso caso tem o nome **sample.csv**. Caso o ficheiro não tenha esse nome os algoritmos não o conseguem ver logo é necessário alterar o nome do **novo ambiente**, feito pelos professores, para isso. <br>
O **resultado** do algoritmo será demonstrado através **de outro ficheiro** ".csv" que será criado na diretória do ficheiro "jupyter notebook", com o nome **result.csv**, e terá dois tipos novos de nós. Estes serão explicados mais à frente. <br>


### Algoritmos
Nesta **solução** iremos utilizar, como solução final, o **algoritmo A*** (A star). No entanto, antes de escolhermos este algoritmo, decidimos testar a sua usabilidade comparado com outros algoritmos de pequisa informada, mais especificamente o **algoritmo de dijkstra**. <br>

Neste relatório iremos explicar a implementação de ambos e o motivo pelo que utilizámos o algoritmo A* (A star). <br>

## Implementação (Dijsktra)

### Imports
Os **imports** feitos inicialmente são partilhados entre os algoritmos. <br>
Um dos imports foi feito para o **pandas**, renomeado para pd, e será utilizado para ler e guardar o ficheiro ".csv". <br>
Outro foi feito para o **csv** e será utilizado na leitura do ficheiro ".csv" em conjunto com o pandas. <br>
O final foi feito para o **heapq**, renomeado para heap, e será utilizado para melhorar a eficiência dos nossos algoritmos na "PriorityQueue" através das ações de adição e remoção. <br>

In [1]:
import pandas as pd
import csv
import heapq as heap

### Funções Adicionais
Antes de iniciarmos a implementação dos algoritmos em si, criámos algumas **funções e classes adicionais** para reduzir a quantidade de código presente e para melhor podermos visualizar a sua implementação. Estas serão utilizadas em ambos os algoritmos. <br>

$\rightarrow$ A **PriorityQueue** foi uma classe construida com base numa queue e que contém dois argumentos, a sua prioridade e a sua posição. A prioridade é utilizada para ordenar a queue enquanto a posição refere-se à posição dos nós na grelha (ambiente). Nesta implementação foi utilizado a bibliteca heap para aumentar a eficiência da queue. <br>

$\rightarrow$ A função **getNeighbors**, como o nome indica, vai buscar todos os filhos à volta de um nó. Estes, como dito na introdução, são todos os nós à sua volta e serão aqueles que se poderá movimentar para. Recebe como argumento o nó atual e retorna todos os seus filhos numa lista. Também verifica se o filho está dentro da grelha e se é um obstáculo. Caso um desses se verifique então esse filho não entra na lista. <br>

$\rightarrow$ A função **getMovementCost** vai verificar a "tag" de um nó, que poderá ser "T" ou "A", e irá retornar um valor com base nisso. Recebe como argumento o nó, que no nosso caso será um dos filhos obtidos através da função anterior, e irá retornar o seu custo de movimento para esse filho. <br>

$\rightarrow$ A função **writeVisited** vai receber a grelha e a lista de custos para escrever nela os **nós** que foram **visitados**. Estes terão a "tag" de **V** e serão todos aqueles que o nosso algoritmo visitou e verificou para serem adicionados. No entanto, a escrita **não** é feita para o **nó inicial** e o **nó final** devido a querermos que estes se mantenham para não causar confusão. Esta escrita tem de ser feita numa função à parte de modo a não estragar a execução do algoritmo. <br>

$\rightarrow$ A função **createResult** vai receber a grelha final e transferi-la para um csv. Para fazer isto utilizámos a biblioteca csv, que foi importada previamente, para escrever no ficheiro "result.csv". Este ficheiro irá conter a resolução do algoritmo na grelha. <br>

In [2]:
#Add Priority Queue class for frontier (contains standard functions)
class PriorityQueue:
    def __init__(self):
        self.elements: [(float, (int, int))] = []
    
    def empty(self):
        return not self.elements
    
    def put(self, item, priority: float):
        heap.heappush(self.elements, (priority, item))
    
    def get(self):
        return heap.heappop(self.elements)[1]

#Get neighbors around current node (fully around)
def getNeighbors(matrix, current, width, height):    
    currentX = current[0]
    currentY = current[1]
    
    #Go through all neighbors
    neighbors = []
    for posX in range(-1, 2):
        for posY in range(-1, 2):
            #Check if we are in current node
            if posX == 0 and posY == 0:
                continue
            
            #Check if we are outside border
            if currentX + posX < 0 or currentX + posX > width - 1:
                continue
            if currentY + posY < 0 or currentY + posY > height - 1:
                continue
            
            #Check if neighbor is obstacle
            neighbor_value = matrix.iloc[currentX + posX, currentY + posY]
            if neighbor_value == "B" or neighbor_value == "F":
                continue
                
            #Add node to neighbors
            neighbors.append((currentX + posX, currentY + posY))
    
    return neighbors

#Get movement cost to neighbor depending on its value
def getMovementCost(matrix, neighbor):
    posX = neighbor[0]
    posY = neighbor[1]
    neighbor_value = matrix.iloc[posX, posY]
    
    #Check neighbor movement for water and earth
    if neighbor_value == "T":
        return 1
    if neighbor_value == "A":
        return 3
    
    #Return 0 if it is neither of those (start or goal)
    return 0

#Write visited nodes to grid
def writeVisited(matrix, cost_so_far):
    #Go through all nodes visited
    for next in cost_so_far:   
        #Dont let it write to start or goal
        if matrix.iloc[next[0], next[1]] == "E" or matrix.iloc[next[0], next[1]] == "S":
            continue
        
        #Write to Matrix
        matrix.iloc[next[0], next[1]] = "V"

#Create the CSV result file
def createResult(matrix):
    #Create header with range
    header = [None] + list(range(0, 100))
    
    #Open file with correct configurations
    with open("result.csv", "w", encoding="UTF8", newline="") as file:
        #Get writer for file
        writer = csv.writer(file)
        
        #Start by writing header row (0 to 99) with None first
        writer.writerow(header)
                                
        #Write rest of rows with number first
        pos = 0
        for row in matrix.values:
            #Create full row with number and row
            full_row = [pos] + list(row)
            
            #Write full row to file
            writer.writerow(full_row)
            pos += 1

### Algoritmo
A função que irá executar o **algoritmo de Dijkstra** irá receber **três argumentos**:

$\rightarrow$ O primeiro é a **matrix**, que é a grelha com o tipo Dataframe. <br> 

$\rightarrow$ O segundo é o **start**, que é o nó inicial onde o algoritmo irá começar. Caso não seja introduzido um valor então começa na posição (0, 0). Isto é devido a ser especificado no enunciado que o nó de partida do algoritmo está no canto superior esquerdo da grelha, no entanto quisemos que o utilizador conseguisse alterar este valor caso esse não fosse o caso. <br>

$\rightarrow$ O terceiro é o **goal**, que é o nó final onde o algoritmo quer chegar. Caso não seja introduzido um valor então começa na posição (99, 99). Isto é devido a ser especificado no enunciado que o nó de chegada do algoritmo está no canto interior direito da grelha, no entanto quisemos que o utilizador conseguisse alterar este valor caso esse não fosse o caso. <br>

Na **inicialização da função**, após a introdução dos argumentos, começámos por criar e inicializar as **variáveis** necessárias para o seu funcionamento: <br>

$\rightarrow$ Estas consistem na **frontier**, que é uma PriorityQueue que contem todos os nós visitados como também a sua prioridade, que é inicializada com o **start**. <br>

$\rightarrow$ Consiste no **width** e no **height**, que serão utilizados para verificar as fronteiras e, embora seja especificado no enunciado que é uma grelha de 100x100, quisemos certificar-nos que o valor estava sempre correto. <br>

$\rightarrow$ Consiste também no **came_from**, que é a ligação entre um nó anterior e o nó para que se move, e no **cost_so_far**, que consiste no custo total até um nó. Estas são as que serão retornadas no fim do algoritmo. <br>

Com a inicialização concluida, continuamos por percorrer o **frontier**. <br>
Assim, começamos por buscar o primeiro nó da queue e verificar se é o **goal**. Caso não seja então vamos buscar os filhos do nó, que é feito através da função **getNeighbors** explicada anteriormente. Como a função não reconhece obstáculos como parte dos filhos não necessitamos de nos preocupar com esses para o resto do algoritmo. <br>

Após termos os filhos do nó vamos continuar por os percorrer. Logo no inicio vamos começar por verificar o **peso do filho** através da função **getMovementCost**. Já tendo o peso, continuamos por efetuar a soma entre o **peso total** até o nó atual com o **peso** do filho. Esta soma será utilizada não só para guardar o **custo total** de movimento para esse filho como também para verificar se é o **caminho de menor custo** para esse nó. Esta verificação será a base que nos permitirá encontrar o caminho de menor custo para o nó final. <br>

Caso a verificação se indique, ou seja que o **filho** não tenha sido **visitado** antes ou que o **custo total** verificado no caminho seja o menor, então vamos adicionar o **filho** à queue, tendo o custo total como prioridade. Também vamos adicioná-lo à lista de custos e à lista de ligações. Depois continua por verificar os outros filhos. <br>

Quando o algoritmo concluir, que é quando a **queue** ficar vazia ou o **goal** for encontrado, retorna-mos a **lista de ligações** e a **lista de custos**. A lista de ligações será necessária para a conclusão do algoritmo no passo final, que será explicado na próxima secção, e a lista de custos será necessária para escrever na grelha os nós que foram **visitados**. <br>

In [3]:
#Implementation of Dijkstra search for grid
def dijkstra_search(matrix, start = (0, 0), goal = (99, 99)):
    #Setup priority queue for node and its priority
    frontier = PriorityQueue()
    frontier.put(start, 0)
    
    #Setup matrix width and height
    width = len(matrix.columns)
    height = len(matrix.values)
    
    #Setup dictionary for cost of each node and for their parents
    came_from: {(int, int), (int, int)} = {}
    cost_so_far: {(int, int), float} = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    #Go through priority queue
    while not frontier.empty():
        #Get current node and check if its goal node
        current = frontier.get()      
        if current == goal:
            break
        
        #Get neighbors of current node
        neighbors = getNeighbors(matrix, current, width, height)
        for next in neighbors:
            #Get movement cost to neighbor
            movement_cost = getMovementCost(matrix, next)
            
            #Use new cost to pick movement
            new_cost = cost_so_far[current] + movement_cost
            if next not in cost_so_far or new_cost < cost_so_far[next]:    
                #Update dictionaries with movement values
                cost_so_far[next] = new_cost
                priority = new_cost
                frontier.put(next, priority)
                came_from[next] = current
    
    #Return list of connections and costs
    return came_from, cost_so_far

### Passo Final
Já tendo a lista de ligações vamos invocar o **passo final** através de uma função. <br>
Esta função tem o nome de **reconstruct_path** e tem como objetivo reconstruir o caminho de uma forma optimizada. Isto vai ser feito através de lista de ligações que foi retornada na execução do algoritmo. <br>

Esta função vai receber como argumentos a **matrix**, o **start** e o **goal**, que servem o mesmo propósito que no algoritmo visto anteriormente, como também o **came_from**, que é a lista de ligações. Depois, inicializamos as variáveis que serão necessárias para a função. Estas sendo o **current**, que é o nó atual que está a ser visto, e o **path**, que é o caminho optimizado que será retornado no fim. <br>

No entanto, a função irá começar a partir do **fim** até chegar ao **inicio**, logo o **current** será inicializado com o **goal**. Assim, percorremos a lista de ligações até o **current** ser igual ao **start**. A **reconstrução** do caminho foi feita através de buscar o **nó conectado** ao **nó atual** (current) e depois tornar esse o nó atual. Devido a termos começado no **fim**, e o algoritmo anterior ter guardado o caminho de menor custo até ele, ao construir o caminho, até chegarmos ao **inicio**, vamos ficar com o **menor custo**. Qualquer posição que seja adicionada nesta parte irá também ser escrita para a grelha com a "tag" **SP**. Isto irá representar o **caminho optimizado** desde o **inicio** até o **fim**. <br>

No fim, após termos encontrado o **nó inicial**, iremos adicioná-lo ao **caminho optimizado**, devido ao algoritmo não efetuar isto normalmente, como também vamos alterar a sua direção para ir desde o **inicio** até ao **fim**. Estas alterações não são necessárias, devido à lista do **caminho optimizado** nem ser utilizada, mas quisemos que o utilizador tivesse outra forma de visualizar o caminho que não fosse a **grelha**. <br>

In [4]:
#Reconstruct path optimally
def reconstruct_path(matrix, came_from, start = (0, 0), goal = (99, 99)):
    current = goal
    path = []
    
    #Go from goal to start in optimal path
    while current != start:
        #Write to Dataframe (dont write over start or goal)
        if current != goal:
            matrix.iloc[current[0], current[1]] = "SP"
        
        #Append to path
        path.append(current)
        current = came_from[current]
        
    #Append start, since previous loop doesnt, and reverse path to go from beginning to end
    path.append(start) # optional
    path.reverse() # optional
    
    #Return optimal path
    return path

### Teste
O **teste** que foi efetuado ao **algoritmo de dijsktra** toma em consideração todos os pontos falados anteriormente para efetuar um **teste** que fosse perceptivel ao utilizador. <br>

Primeiro começamos por ler o ficheiro **csv**. Devido a um **erro** na leitura do ficheiro, que adiciona uma **coluna** a mais no **dataframe**, somos forçados a remove-la para podermos efetuar a **leitura** correta dos dados. <br>

Depois demonstramos a grelha antes e depois de ser efetuado o **algoritmo de djisktra** nela. Para poder ser visualizado invocamos a função **writeVisited** depois de efetuarmos o algoritmo. Como pode ser visto, foi adicionado a "tag" **V** que representa os nós que foram visitados pelo algoritmo. No caso do **algoritmo de dijsktra** ele visualiza todos os nós da grelha, sem ser os obstáculos e o inicial e final, logo maior parte estão com essa "tag". Também é demonstrado o **custo total** para chegar ao **nó final**, depois de executar o algoritmo, que no nosso caso é 164. <br>

Depois efetuamos o passo final, sendo esse a **reconstrução** do **caminho optimizado** com base nos nós visitados, e demonstramos a grelha com esse caminho. Como pode ser visto, foi adicionada a "tag" **SP** que representa o **caminho optimizado**. <br>

Já tendo concluido a pesquisa vamos escrever a **grelha final**, que contém os nós visitados e o caminho mais curto, para o ficheiro **csv** com o nome "result.csv". <br>

In [5]:
#Start code with message
print("Using informed search algorithm (dijkstra) to find shortest route to goal from start")

#Read grid from CSV file
data = pd.read_csv("sample.csv")

#Remove column with column numbers as Dataframe already has it (extra column)
del data[data.columns[0]]

#Show initial grid
print("Initial Grid")
display(data)

#Activate search algorithm for grid
came_from, cost_so_far = dijkstra_search(matrix = data)

#Show total cost to reach goal
print("Total Cost to reach goal: ")
display(cost_so_far[(99, 99)])

#Write visited nodes in grid (will replace values seen with "V")
writeVisited(data, cost_so_far)

#Show grid with replaced values
display("Grid with Visited nodes (V)")
display(data)

#Optimize values seen which will replace values seen with "SP"
path = reconstruct_path(data, came_from)

#Show grid with final values
print("Grid with Shortest Path (SP)")
display(data)

#Save grid in new CSV with name "result.csv"
createResult(data)

#Conclude code with message
print("Successfully searched through Grid for goal (dijsktra)!")

Using informed search algorithm (dijkstra) to find shortest route to goal from start
Initial Grid


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,S,F,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
1,F,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,T,F
2,F,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,A,F
3,F,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,A,F
4,F,T,T,T,T,T,T,T,T,T,...,T,T,A,A,A,A,A,A,A,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,F,T,T,T,T,T,T,A,A,A,...,A,T,T,T,A,A,A,A,T,F
96,F,T,T,T,T,T,T,A,A,A,...,T,T,A,A,A,A,A,T,T,F
97,F,T,T,T,T,T,T,T,T,T,...,T,T,A,A,A,A,A,T,T,F
98,F,T,T,T,T,T,T,T,T,T,...,T,T,A,A,A,T,T,T,T,F


Total Cost to reach goal: 


164

'Grid with Visited nodes (V)'

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,S,F,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
1,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F
2,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F
3,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F
4,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F
96,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F
97,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F
98,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F


Grid with Shortest Path (SP)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,S,F,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
1,F,SP,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F
2,F,SP,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F
3,F,SP,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F
4,F,SP,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,V,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,SP,F
96,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,SP,V,F
97,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,SP,V,F
98,F,V,V,V,V,V,V,V,V,V,...,V,V,V,V,V,V,V,V,SP,F


Successfully searched through Grid for goal (dijsktra)!


### Resultado
Como pode ser visto, na **execução do teste**, o **algoritmo de dijsktra**, embora funcional, demora muito tempo a efetuar a sua pesquisa. Isto é devido ao número de nós que têm de visitar, sendo que o algoritmo visita quase todos os nós da grelha à procura do caminho mais curto. <br>

Na execução do próximo código pode ser visto que o **algoritmo de dijsktra** visita uma grande quantidade de nós. <br>

Deste modo nós pensámos noutro algoritmo que nos pudesse encontrar o **caminho mais curto** de uma forma mais rápida. Isto levou-nos ao **algoritmo A*** que tinha sido dado nas aulas e que era muito parecido com o **algoritmo de dijsktra** sem ser numa **diferença**. <br>

In [6]:
#Get number of nodes visited by dijsktra
nodes_visited_dijsktra = len(came_from)

#Show number of nodes visited
print("Total number of nodes visited: (Dijsktra)")
display(nodes_visited_dijsktra)

Total number of nodes visited: (Dijsktra)


8248

## Implementação (A*)
A **diferença** entre o algoritmo de dijsktra e o algoritmo A* está toda na **heurística**. <br>
Isto será o foco desta secção devido ao código, sem ser a adição da **heurística**, ser exatamente igual ao **algoritmo de dijsktra**. <br>

### Heurística
Uma **heurística** é uma estimativa, mais especificamente **our very best educated guess**, e dá nos uma estimativa do custo, ou esforço, para chegar ao **nó final** a partir do **nó atual**. <br>

No inicio, devido a ser uma das heurísticas que nós demos nas aulas, decidimos usar a **manhattan distance**. <br>
Esta é uma das heurísticas **standard** e o que faz é que olha para a soma da distância absoluta **horizontal** e **vertical** da posição do **nó atual** até o **nó final**. No entanto, com um pouco de pesquisa, descobrimos que esta heurística é melhor para quando só é possivel movimentar-se em **4 direções**, que não é o caso para este algoritmo que é suposto movimentar-se em **8 direções**. Para esse caso, a melhor **heurística** é a **diagonal distance**. <br>

A **diagonal distance** toma em consideração o movimento diagonal no calculo da distãncia. <br>
Desse modo, o que faz é que tomamos em consideração o número de passos que pode-se tomar se não podermos andar diagonalmente. Depois, subtraimos a este número os passos que podemos tomar, e salvar, se andarmos diagonalmente. Também temos de tomar em consideração os **argumentos extra**, sendo esses o **D** e o **D2**. O **D** será o custo de nos movermos na horizontal e na vertical enquanto o **D2** será o custo de nos movermos na diagonal. No nosso caso ambos destes valores serão igual a **1** devido a não haver diferença na **distância absoluta**. Isto também é conhecido como a **chebyshev distance**. <br>

In [7]:
#Manhattan Distance (Base heuristic, works fine but could be better)
def heuristic_manhattan(current, goal):
    (x1, y1) = current
    (x2, y2) = goal
    
    return abs(x1 - x2) + abs(y1 - y2)

#Diagonal Distance (Improved version of manhattan, works better with diagonal movement)
def heuristic_diagonal(current, goal):
    (x1, y1) = current
    (x2, y2) = goal
    
    #Define extra arguments (Chebyshev distance)
    D = 1
    D2 = 1
    
    dx = abs(x1 - x2)
    dy = abs(y1 - y2)
    return D * (dx + dy) + (D2 - 2 * D) * min(dx, dy)

### Algoritmo
Como dito previamente, o **algoritmo A*** é praticamente igual ao **algoritmo de dijsktra**. A única diferença entre estes é a adição da **heurística**. Esta será adicionada na prioridade, da **PriorityQueue**, para escolher o próximo nó que iremos movimentar para. <br>

A **heurística** utilizada neste algoritmo foi a **diagonal distance** pelas razões dadas previamente. <br>

In [8]:
#Implementation of A* search for grid
def a_star_search(matrix, start = (0, 0), goal = (99, 99)):
    #Setup priority queue for node and its priority
    frontier = PriorityQueue()
    frontier.put(start, 0)
    
     #Setup matrix width and height
    width = len(matrix.columns)
    height = len(matrix.values)
    
    #Setup dictionary for cost of each node and for their parents
    came_from: {(int, int), (int, int)} = {}
    cost_so_far: {(int, int), float} = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    #Go through priority queue
    while not frontier.empty():
        #Get current node and check if its goal node
        current = frontier.get()
        if current == goal:
            break
        
        #Get neighbors of current node
        neighbors = getNeighbors(matrix, current, width, height)
        for next in neighbors:
            #Get movement cost to neighbor
            movement_cost = getMovementCost(matrix, next)
            
            #Use new cost to pick movement
            new_cost = cost_so_far[current] + movement_cost
            if next not in cost_so_far or new_cost < cost_so_far[next]:            
                #Update dictionaries with movement values
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic_diagonal(next, goal)
                frontier.put(next, priority)
                came_from[next] = current
    
    #Return list of connections and costs
    return came_from, cost_so_far

### Teste
O **teste** que foi efetuado ao **algoritmo A*** é igual ao teste efetuado ao **algoritmo de dijkstra**, sem ser no algoritmo utilizado. <br>

Na visualização da grelha, depois de ter sido efetuado o **algoritmo A***, pode-se verificar que os nós **visitados** são mais representados na direção do **goal** e já não percorrem a grelha inteira tal como tinha sido visto anteriormente no **algoritmo de dijkstra**. <br>

O resultado deste algoritmo irá também ser escrito no mesmo ficheiro **csv**, removendo o **algoritmo de dijsktra** caso esse tenha sido feito anteriormente. <br>

In [10]:
#Start code with message
print("Using informed search algorithm (A*) to find shortest route to goal from start")

#Read grid from CSV file
data = pd.read_csv("sample.csv")

#Remove column with column numbers as Dataframe already has it (extra column)
del data[data.columns[0]]

#Show initial grid
print("Initial Grid")
display(data)

#Activate search algorithm for grid
came_from, cost_so_far = a_star_search(matrix = data)

#Show total cost to reach goal
print("Total Cost to reach goal: ")
display(cost_so_far[(99, 99)])

#Write visited nodes in grid (will replace values seen with "V")
writeVisited(data, cost_so_far)

#Show grid with replaced values
print("Grid with Visited nodes (V)")
display(data)

#Optimize values seen which will replace values seen with "SP"
reconstruct_path(data, came_from)

#Show grid with final values
print("Grid with Shortest Path (SP)")
display(data)

#Save grid in new CSV with name "result.csv"
createResult(data)

#Conclude code with message
print("Successfully searched through Grid for goal (A*)!")

Using informed search algorithm (A*) to find shortest route to goal from start
Initial Grid


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,S,F,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
1,F,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,T,F
2,F,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,A,F
3,F,T,T,T,T,T,T,T,T,T,...,T,T,T,T,T,A,A,A,A,F
4,F,T,T,T,T,T,T,T,T,T,...,T,T,A,A,A,A,A,A,A,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,F,T,T,T,T,T,T,A,A,A,...,A,T,T,T,A,A,A,A,T,F
96,F,T,T,T,T,T,T,A,A,A,...,T,T,A,A,A,A,A,T,T,F
97,F,T,T,T,T,T,T,T,T,T,...,T,T,A,A,A,A,A,T,T,F
98,F,T,T,T,T,T,T,T,T,T,...,T,T,A,A,A,T,T,T,T,F


Total Cost to reach goal: 


164

Grid with Visited nodes (V)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,S,F,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
1,F,V,V,V,V,V,V,V,V,V,...,T,T,T,T,T,A,A,A,T,F
2,F,V,V,V,V,V,V,V,V,V,...,T,T,T,T,T,A,A,A,A,F
3,F,V,V,V,V,V,V,V,V,V,...,T,T,T,T,T,A,A,A,A,F
4,F,V,V,V,V,V,V,V,V,V,...,T,T,A,A,A,A,A,A,A,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,F,T,T,T,T,T,T,A,A,A,...,V,V,V,V,V,V,V,V,V,F
96,F,T,T,T,T,T,T,A,A,A,...,T,V,V,V,V,V,V,V,V,F
97,F,T,T,T,T,T,T,T,T,T,...,T,T,A,A,A,A,V,V,V,F
98,F,T,T,T,T,T,T,T,T,T,...,T,T,A,A,A,T,V,V,V,F


Grid with Shortest Path (SP)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,S,F,F,F,F,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
1,F,SP,V,V,V,V,V,V,V,V,...,T,T,T,T,T,A,A,A,T,F
2,F,V,SP,V,V,V,V,V,V,V,...,T,T,T,T,T,A,A,A,A,F
3,F,V,V,SP,V,V,V,V,V,V,...,T,T,T,T,T,A,A,A,A,F
4,F,V,V,V,SP,V,V,V,V,V,...,T,T,A,A,A,A,A,A,A,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,F,T,T,T,T,T,T,A,A,A,...,V,V,V,V,V,V,V,V,SP,F
96,F,T,T,T,T,T,T,A,A,A,...,T,V,V,V,V,V,V,SP,V,F
97,F,T,T,T,T,T,T,T,T,T,...,T,T,A,A,A,A,V,SP,V,F
98,F,T,T,T,T,T,T,T,T,T,...,T,T,A,A,A,T,V,V,SP,F


Successfully searched through Grid for goal (A*)!


### Resultado
Como pode ser visto, na **execução do teste**, o **algoritmo A*** é muito mais rápido que o **algoritmo de dijkstra**. Isto é devido a não necessitar de percorrer todos os nós da grelha para encontrar o caminho de menor custo. No caso do **A*** só percorre os nós que estão na "direção" para o **goal**. <br>

Na execução do próximo código conseguimos **comparar** os nós visitados entre os dois algoritmos. Como pode ser visto, o **algoritmo A*** visita um número menor que o **algoritmo de dijsktra** e, desse modo, vai ser o mais eficiente. <br>

Desse mesmo modo, pode-se reparar que o **caminho mais curto** feito pelo **algoritmo A*** é um pouco diferente ao do **algoritmo de dijsktra** embora tenham o mesmo custo. Esta **diferença**, no entanto, só acontece no ínicio do algoritmo e é devido a haver dois caminhos que estão **empatados** em termos de custo no ínicio. Desse modo não há nenhum problema nesta diferença. <br>

Assim, ao comparar os dois algoritmos, decidimos que o **A*** era melhor que o **dijsktra**, sendo mais rápido que o **dijsktra** devido a visitar menos nós, e seria o que iriamos utilizar para encontrar o caminho válido mais curto no **novo ambiente** dado pelos professores. <br>

In [11]:
#Get number of nodes visited by A*
nodes_visited_astar = len(came_from)

#Show number of nodes visited
print("Total number of nodes visited: (A*)")
display(nodes_visited_astar)

#Compare number of nodes visited between both algorithms
print("Compare number of nodes visited between Dijsktra and A* (In this order): " + str(nodes_visited_dijsktra) + " to " + str(nodes_visited_astar))

Total number of nodes visited: (A*)


5570

Compare number of nodes visited between Dijsktra and A* (In this order): 8248 to 5570


## Conclusão
O **algoritmo A*** demonstrou-se mais que capaz de encontrar o caminho válido mais curto de uma forma eficaz logo estamos felizes com o resultado final do nosso projeto. Isto não veio sem desafios, devido a procurar-mos uma implementação dos vários algoritmos que pudessemos alterar para funcionar com o que necessitavamos como também da informação necessária para faze-lo o mais eficaz possivel. <br>

No entanto, estes desafios levaram-nos a entender melhor o tópico de pesquisa informada, especialmente em grelhas, e que nos levou à implementação atual do A*. <br>